<a href="https://colab.research.google.com/github/anadiedrichs/2023-vision-computacional/blob/main/TP3_vision_Diedrichs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Trabajo Práctico Nro 3


Ana Laura Diedrichs

SIU a0902

9Co2022


 Encontrar el logotipo de la gaseosa dentro de las imágenes provistas en Material_TPs/TP3/images a partir del template Material_TPs/TP3/template




In [ ]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
%matplotlib inline
# OpenCV-Python utiliza NumPy para el manejo de imágenes
import numpy as np
# cv2 es el módulo python para acceder a OpenCV
import cv2 as cv
# matplotlib para mostrar imágenes, perfiles, histogramas, etc
import matplotlib.pyplot as plt
# para importar imagenes desde urls
from skimage import io
# mostrar imagenes opencv en googlecolab
from google.colab.patches import cv2_imshow

Muestro la imagen del template

In [ ]:
url_template = "https://www.dropbox.com/s/rn5d4oubths7xmz/pattern.png?dl=1"
img = io.imread(url_template)
img_template = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

plt.figure(figsize=(8,8))
plt.imshow(img_template, cmap='gray')

## 1.(4 puntos) Obtener una detección del logo en cada imagen sin falsos positivos


In [ ]:
def template_matching(imagen, plantilla, t = 0.7):
  'Recibe imágenes y se puede elegir el threshold / umbral'
  # Convert the images to grayscale
  copia = imagen.copy()
  input_gray = cv.cvtColor(imagen, cv.COLOR_BGR2GRAY)
  template_gray = cv.cvtColor(plantilla, cv.COLOR_BGR2GRAY)

  # Find the dimensions of the template image
  h,w = template_gray.shape

  # Use the cv2.matchTemplate() function to find the correlation between the images
  result = cv.matchTemplate(input_gray, template_gray, cv.TM_CCOEFF_NORMED)

  # Set the threshold for the correlation coefficient
  threshold = t
  loc = np.where(result >= threshold)

  # Check if there are any good matches
  if len(loc[0]) > 0:
      # Draw a rectangle around each good match
      for pt in zip(*loc[::-1]):
          cv.rectangle(copia, pt, (pt[0] + w, pt[1] + h), (0, 0, 255), 2)
      print("Coincidencia encontrada")
  else:
      print("Nada por aquí")

  # Find the location of the maximum value in the result image
  min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)

  # Draw a rectangle around the maximum value in the result image
  top_left = max_loc
  bottom_right = (top_left[0] + plantilla.shape[1], top_left[1] + plantilla.shape[0])
  img_matches = cv.rectangle(copia, top_left, bottom_right, (0, 0, 255), 2)

  return img_matches




In [ ]:
def template_matching_intro(imagen,plantilla):

    img_gray = cv.cvtColor(imagen, cv.COLOR_BGR2GRAY)
    template = cv.cvtColor(plantilla, cv.COLOR_BGR2GRAY)

    # Hago una copia de la imagen porque ciclo a ciclo le dibujo rectángulos
    img_salida =  cv.cvtColor(imagen, cv.COLOR_BGR2RGB)

    h,w = template.shape
    # Aplicamos la coincidencia de patrones
    #--------------------------------------
    res = cv.matchTemplate(img_gray, template, cv.TM_SQDIFF_NORMED)

    # Encontramos los valores máximos y mínimos
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)

    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    #if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
    top_left = min_loc
    #else:
    #    top_left = max_loc

    # Marcamos el lugar donde lo haya encontrado
    #----------------------------------------
    bottom_right = (top_left[0] + w, top_left[1] + h)
    cv.rectangle(img_salida,top_left, bottom_right, 255, 2)

    # Graficamos el procesamiento y la salida
    #----------------------------------------
    plt.figure()

    # Resultado de coincidencia
    plt.subplot(121),plt.imshow(res,cmap = 'gray')
    plt.title('Matching Result'), plt.xticks([]), plt.yticks([])

    # Imagen original con recuadros
    plt.subplot(122),plt.imshow(img_salida)
    plt.title('Detected Point'), plt.xticks([]), plt.yticks([])

    plt.suptitle('TM_CCOEFF_NORMED')
    plt.show()

### Imagen COCA-COLA-LOGO.jpg

In [ ]:
# template
url_template = "https://www.dropbox.com/s/rn5d4oubths7xmz/pattern.png?dl=1"
img = io.imread(url_template)
# imagen
url = "https://www.dropbox.com/s/lxv9drjd0vzmzg2/COCA-COLA-LOGO.jpg?dl=1"
myImg = io.imread(url)

In [ ]:
template_matching_intro(myImg, plantilla=img)

In [ ]:

salida = template_matching(myImg, plantilla=img, t = 0.6)

# Display the results
plt.figure(figsize = (20,10))
plt.imshow(salida)

### Imagen coca_logo_1.png

In [ ]:
# imagen
url = "https://www.dropbox.com/s/bifxxgy5l0vicf0/coca_logo_1.png?dl=1"
myImg = io.imread(url)
#template_matching_intro(myImg, plantilla=img)

La ejecución anterior da error porque la resolución de la imagen coca_logo_1.png es menor a la del template, corrijo achicando la imagen del template con pyrDown

In [ ]:
# Esto aumenta la resolución inyectando filas y columnas de ceros intercaladas y luego
# convolucionando el resultado con el núcleo gaussiano 5 × 5 multiplicado por 4
img_nivel_2 = cv.pyrDown(img)

# Muestro el resultado por subir el nivel
#cv2_imshow(img_nivel_2)
template_matching_intro(myImg, plantilla=img_nivel_2)

In [ ]:
salida = template_matching(myImg, plantilla=img_nivel_2, t = 0.9)

# Display the results
plt.figure(figsize = (20,10))
plt.imshow(salida)

### Modificación al preprocesamiento de las imágenes previo a aplicar template matching



In [ ]:
# load the image image, convert it to grayscale, and detect edges
# template
url_template = "https://www.dropbox.com/s/rn5d4oubths7xmz/pattern.png?dl=1"
img = io.imread(url_template)
template = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
template = cv.Canny(template, 50, 200)
(tH, tW) = template.shape[:2]
cv2_imshow( template)

In [ ]:
urls = ["https://www.dropbox.com/s/lxv9drjd0vzmzg2/COCA-COLA-LOGO.jpg?dl=1",
     "https://www.dropbox.com/s/bifxxgy5l0vicf0/coca_logo_1.png?dl=1",
      "https://www.dropbox.com/s/a4y0lsjykqzmm7b/coca_logo_2.png?dl=1",
      "https://www.dropbox.com/s/o1rtc9zsu81u0eq/coca_retro_1.png?dl=1",
      "https://www.dropbox.com/s/jgxf1g1v94wdw6r/coca_retro_2.png?dl=1",
      "https://www.dropbox.com/s/pfwbo9nnyi5ifyq/logo_1.png?dl=1",
      "https://www.dropbox.com/s/z0sfx2k08ou61at/coca_multi.png?dl=1"]


In [ ]:
import imutils
visualize = False

# loop over the images to find the template in
for imagePath in urls:
  # load the image, convert it to grayscale, and initialize the
  # bookkeeping variable to keep track of the matched region
  image = io.imread(imagePath)
  gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
  found = False
  # loop over the scales of the image
  for scale in np.linspace(0.2, 1.0, 10)[::-1]: # el 10 antes era 20
    # resize the image according to the scale, and keep track
    # of the ratio of the resizing
    resized = imutils.resize(gray, width = int(gray.shape[1] * scale))
    r = gray.shape[1] / float(resized.shape[1])

    # if the resized image is smaller than the template, then break
    # from the loop, para evitar error en opencv
    if resized.shape[0] < tH or resized.shape[1] < tW:
      break
    found = True
    # detect edges in the resized, grayscale image and apply template
    # matching to find the template in the image
    edged = cv.Canny(resized, 50, 200)
    result = cv.matchTemplate(edged, template, cv.TM_CCOEFF)
    _, maxVal, _, maxLoc = cv.minMaxLoc(result)

    # check to see if the iteration should be visualized
    if( visualize == True):
      # draw a bounding box around the detected region
      clone = np.dstack([edged, edged, edged])
      cv.rectangle(clone, (maxLoc[0], maxLoc[1]),
        (maxLoc[0] + tW, maxLoc[1] + tH), (0, 0, 255), 2)
      cv2_imshow( clone)

    (startX, startY) = (int(maxLoc[0] * r), int(maxLoc[1] * r))
    (endX, endY) = (int((maxLoc[0] + tW) * r), int((maxLoc[1] + tH) * r))
    # draw a bounding box around the detected result and display the image
    icopia = image.copy()
    cv.rectangle(icopia, (startX, startY), (endX, endY), (0, 0, 255), 2)
    cv2_imshow(icopia)


##2.(4 puntos) Plantear y validar un algoritmo para múltiples detecciones en la imagen
coca_multi.png con el mismo template del ítem 1


In [ ]:
coca_multi_path = "https://www.dropbox.com/s/z0sfx2k08ou61at/coca_multi.png?dl=1"

In [ ]:
# load the image image, convert it to grayscale, and detect edges
# template
url_template = "https://www.dropbox.com/s/rn5d4oubths7xmz/pattern.png?dl=1"
img = io.imread(url_template)
template = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
template = cv.Canny(template, 50, 200)
(tH, tW) = template.shape[:2]
cv2_imshow( template)

In [ ]:
template.shape[:2]

In [ ]:
import imutils
from imutils.object_detection import non_max_suppression

visualize = True
thresh = 0.1

In [ ]:
# load the image, convert it to grayscale, and initialize the
# bookkeeping variable to keep track of the matched region
image = io.imread(coca_multi_path)
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
color = cv.cvtColor(image, cv.COLOR_BGR2RGB)
found = False
# loop over the scales of the template
for scale in np.linspace(0.2, 1.0, 10)[::-1]: # el 10 antes era 20
  # resize the image according to the scale, and keep track
  # of the ratio of the resizing


  template_resized = imutils.resize(template, width = int(template.shape[1] * scale))
  (tH, tW) = template_resized.shape[:2]

  #resized = imutils.resize(gray, width = int(gray.shape[1] * scale))
  r = template_resized.shape[1] / float(template_resized.shape[1])

  # if the resized image is smaller than the template, then break
  # from the loop, para evitar error en opencv
  if gray.shape[0] < tH or gray.shape[1] < tW:
    if( visualize == True):
      print("NOT FOUND Res. "+ str(scale) + " Image "+imagePath)
    break

  # detect edges in the resized, grayscale image and apply template
  # matching to find the template in the image
  edged = cv.Canny(gray, 50, 200)
  result = cv.matchTemplate(edged, template_resized, cv.TM_SQDIFF)

  _, maxVal, _, maxLoc = cv.minMaxLoc(result)
  #print(maxVal, maxVal)
   # check to see if the iteration should be visualized
  if( visualize == True):
    # draw a bounding box around the detected region
    clone = np.dstack([edged, edged, edged])
    cv.rectangle(clone, (maxLoc[0], maxLoc[1]),
      (maxLoc[0] + tW, maxLoc[1] + tH), (0, 0, 255), 2)
    cv2_imshow( clone)


  # Define a threshold
  # thresh between 40000000 and 60000000 works
  thresh = 50000000

  # Select rectangles with confidence less than threshold for TM_SQDIFF
  (y_points, x_points) = np.where(result <= thresh)

  # Select rectangles with
  # confidence greater than threshold
  (y_points, x_points) = np.where(result >= thresh)

  # initialize our list of rectangles
  boxes = list()

  # loop over the starting (x, y)-coordinates again
  for (x, y) in zip(x_points, y_points):
      # update our list of rectangles
      boxes.append((x, y, x + tW, y + tH))
  print("hay "+str(len(boxes))+ " cajitas")
  # apply non-maxima suppression to the rectangles
  # this will create a single bounding box para evitar que se repitan o superpongan rectangulos
  boxes = non_max_suppression(np.array(boxes))
  copia = color.copy()
  # loop over the final bounding boxes
  for (x1, y1, x2, y2) in boxes:
      # draw the bounding box on the image
      cv.rectangle(copia, (x1, y1), (x2, y2),(255, 0, 0), 3)

  cv2_imshow(copia)




##3.(2 puntos) Generalizar el algoritmo del item 2 para todas las imágenes.
Visualizar los resultados con bounding boxes en cada imagen mostrando el nivel de confianza
de la detección.


In [ ]:
template_image = img_template

height, width = template_image.shape

for url in urls: # por cada imagen
  myImg = io.imread(url)
  img_rgb= cv.cvtColor(myImg, cv.COLOR_BGR2RGB)
  target_image= cv.cvtColor(myImg, cv.COLOR_BGR2GRAY)

  # Realizar el template matching
  result = cv.matchTemplate(target_image, template_image, cv.TM_CCOEFF_NORMED)

  # Establecer un umbral para la detección
  threshold = 0.70
  loc = np.where(result >= threshold)

  img_salida = img_rgb.copy()

  for pt in zip(*loc[::-1]):
    cv.rectangle(img_salida, pt, (pt[0] + w, pt[1] + h), (0,0,255), 2)

  # Graficamos el procesamiento y la salida
  #----------------------------------------
  plt.figure()

  # Resultado de coincidencia
  plt.subplot(121),plt.imshow(res,cmap = 'gray')
  plt.title('Matching Result'), plt.xticks([]), plt.yticks([])

  # Imagen original con recuadros
  plt.subplot(122),plt.imshow(img_salida)
  plt.title('Detected Point'), plt.xticks([]), plt.yticks([])
  plt.show()


